In [1]:
import pickle

import keras.preprocessing.sequence as k_ps
import keras.models as k_mod
import keras.layers as k_lay
import keras.optimizers as k_opt
import keras.utils as k_ut
import numpy as np
import pandas as pd
import sklearn.metrics as sk_mt
import sklearn.model_selection as sk_ms
import sklearn.preprocessing as sk_pp
import sklearn.utils as sk_ut

Using TensorFlow backend.


In [2]:
# https://drive.google.com/open?id=1sFf7QUvHwbYSsFEDRRTzxNm_WsnoBU2x
with open('lstm_tweets_net.pkl', 'rb') as f:
    data = pickle.load(f)
x, y = zip(*data)

In [3]:
pad_tag = '#PAD#'
max_len = max(len(xw) for xw in x)

In [4]:
flat_tmp_x = set([b for a in x for b in a])

indexed_tmp_x = {w: i + 1 for i, w in enumerate(flat_tmp_x)}
indexed_tmp_x[pad_tag] = 0
tagged_x = np.array([[indexed_tmp_x[wx] for wx in sx] for sx in x])

lb = sk_pp.LabelBinarizer()
lb.fit(y)

x_tagged_train, x_tagged_test, y_train, y_test = sk_ms.train_test_split(tagged_x, lb.transform(y), shuffle=True)

In [5]:
input_dim = len(x[0])
output_dim = len(set(y))

In [6]:
model = k_mod.Sequential()
model.add(k_lay.Embedding(len(flat_tmp_x) + 1, 256, input_length=max_len))
model.add(k_lay.LSTM(128))
model.add(k_lay.Dense(output_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=k_opt.Adam(0.001), metrics=['accuracy'])
model.fit(x_tagged_train, y_train, batch_size=128, epochs=20)

Epoch 1/20
14133/14133 [==============================] - 23s 2ms/step - loss: 1.2499 - acc: 0.4043
Epoch 2/20
14133/14133 [==============================] - 21s 2ms/step - loss: 0.9358 - acc: 0.6135
Epoch 3/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.6960 - acc: 0.7221
Epoch 4/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.5501 - acc: 0.7764
Epoch 5/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.4649 - acc: 0.8104
Epoch 6/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.4094 - acc: 0.8323
Epoch 7/20
14133/14133 [==============================] - 23s 2ms/step - loss: 0.3640 - acc: 0.8496
Epoch 8/20
14133/14133 [==============================] - 21s 2ms/step - loss: 0.3283 - acc: 0.8637
Epoch 9/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.3104 - acc: 0.8703
Epoch 10/20
14133/14133 [==============================] - 22s 2ms/step - loss: 0.2818 - acc: 0.8818

In [7]:
pred = model.predict_classes(x_tagged_test)
y_pred = lb.inverse_transform(k_ut.to_categorical(pred))

In [8]:
print(sk_mt.classification_report(lb.inverse_transform(y_test), y_pred))

              precision    recall  f1-score   support

      actors       0.36      0.38      0.37      1185
     athlets       0.42      0.41      0.41      1200
    musicans       0.40      0.42      0.41      1139
  politicans       0.62      0.56      0.58      1188

   micro avg       0.44      0.44      0.44      4712
   macro avg       0.45      0.44      0.44      4712
weighted avg       0.45      0.44      0.44      4712

